In [1]:
import os
import re
import numpy as np
import seaborn as sns
import pandas as pd
import string
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, precision_score, recall_score, accuracy_score, balanced_accuracy_score, ConfusionMatrixDisplay
from nltk.stem.porter import PorterStemmer
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1125)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1125)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1125)>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify
[nltk_data]     failed: unable to get local issuer certificate
[nltk_data]     (_ssl.c:1125)>


False

In [3]:
import ssl
import nltk

# Bypass SSL verification temporarily
ssl._create_default_https_context = ssl._create_unverified_context

# Download the necessary data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to /Users/lequochuy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lequochuy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lequochuy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/lequochuy/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
import nltk
import ssl
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
#from nltk.tokenize import word_tokenize, pos_tag
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.feature_selection import SelectKBest, chi2

In [5]:
path_prepost = "/Users/lequochuy/Documents/Research/UIT/bài toán 2"
# path_data = os.path.join(root,"PROJECTS/EDUCATION/DATA")

In [6]:
def ReadExcel(original,dirt):
  answer_path = os.path.join(original,dirt)
  answer = pd.read_excel(answer_path)
  return answer

In [7]:
def ReadCSV(original,dirt):
  answer_path = os.path.join(original,dirt)
  answer = pd.read_csv(answer_path)
  return answer

In [8]:
def HistogramData(df_data,namefields):
  for name in namefields:
    n, bins = np.histogram(df_data[name])
    plt.hist(df_data[name],bins)
    print(bins)
    plt.show()

In [9]:
def AddCountField(df_data):
  df_res = df_data.copy()
  for name, group in df_res.groupby('Mã số ID'):
    #print(f"Group: {name,len(group)}")
    # Process elements within the group (e.g., print, calculate statistics)
    df_res.loc[df_res['Mã số ID'] == name,'count'] = range(1, (df_res['Mã số ID'] == name).sum()+1)#/(df_res['Mã số ID'] == name).sum()
  return df_res

In [10]:
def AssignLabel(df_data):
  df_data.loc[(df_data['Q. 1 /10.00'] >= 9) & (df_data['Q. 1 /10.00'] <=10),'Q. 1 /10.00'] = 10
  df_data.loc[(df_data['Q. 1 /10.00'] >= 8) & (df_data['Q. 1 /10.00'] < 9),'Q. 1 /10.00'] = 9
  df_data.loc[(df_data['Q. 1 /10.00'] >= 7) & (df_data['Q. 1 /10.00'] < 8),'Q. 1 /10.00'] = 8
  df_data.loc[(df_data['Q. 1 /10.00'] >= 6) & (df_data['Q. 1 /10.00'] < 7),'Q. 1 /10.00'] = 7
  df_data.loc[(df_data['Q. 1 /10.00'] >= 5) & (df_data['Q. 1 /10.00'] < 6),'Q. 1 /10.00'] = 6
  df_data.loc[(df_data['Q. 1 /10.00'] >= 4) & (df_data['Q. 1 /10.00'] < 5),'Q. 1 /10.00'] = 5
  df_data.loc[(df_data['Q. 1 /10.00'] >= 3) & (df_data['Q. 1 /10.00'] < 4),'Q. 1 /10.00'] = 4
  df_data.loc[(df_data['Q. 1 /10.00'] >= 2) & (df_data['Q. 1 /10.00'] < 3),'Q. 1 /10.00'] = 3
  df_data.loc[(df_data['Q. 1 /10.00'] >= 1) & (df_data['Q. 1 /10.00'] < 2),'Q. 1 /10.00'] = 2
  df_data.loc[df_data['Q. 1 /10.00'] < 1,'Q. 1 /10.00'] = 1

In [11]:
def ReadLabFiles(original,df_files):
  df_labs = pd.DataFrame()
  for idx, row in df_files.iterrows():
    ans = ReadCSV(original,row["content"])
    ans = AddCountField(ans)

    print(ans.shape)

    df_labs = pd.concat([df_labs, ans], ignore_index=True)
    print(df_labs.shape)

    #print(ans.loc[0:5,['Mã số ID','Điểm/10.00']])
  return df_labs

In [12]:
def GetStudentInfo(df_data, IDST):
  return df_data[df_data['Mã số ID'] == IDST]

In [13]:
def right_substring_to_char(s, leftchar, rightchar):
  """Returns the right substring of s starting from the last occurrence of char.

  Args:
    s: The string to extract the substring from.
    char: The character to find the starting index from.

  Returns:
    The right substring of s starting from the last occurrence of char,
    or the entire string if char is not found.
  """
  last_index = s.rfind('/')
  dir = s[:last_index+1]
  left_index = s.rfind(leftchar)
  if left_index != -1:
    sleft = s[left_index + 1:]
    right_index = sleft.rfind(rightchar)
    return dir + sleft[:right_index] + '.txt'
  else:
    return s

In [14]:
def ReadTextFromExcel(original,dirt):
  token_dict = {}
  text = ""
  fexcel = ReadExcel(original,dirt)
  for idx, row in fexcel.iterrows():
    path_row = row['content']
    ftext = right_substring_to_char(path_row,'[',']')
    print(ftext)
    with open (os.path.join(original,ftext)) as content:
      text = content.read()
      text = text_clean(text)
      token_dict[ftext] = text
  return token_dict


In [15]:
def text_clean(text):
    text = re.sub(r"\n"," ",text)   #remove line breaks
    text = text.lower() #convert to lowercase
    text = re.sub(r"\d+","",text)   #remove digits and currencies
    text = re.sub(r'[\$\d+\d+\$]', "", text)
    text = re.sub(r'[()<>::]', " ", text)
    text = re.sub(r'\d+[\.\/-]\d+[\.\/-]\d+', '', text)   #remove dates
    text = re.sub(r'\d+[\.\/-]\d+[\.\/-]\d+', '', text)
    text = re.sub(r'\d+[\.\/-]\d+[\.\/-]\d+', '', text)
    text = re.sub(r'[^\x00-\x7f]',r' ',text)   #remove non-ascii
    text = re.sub(r'[^\w\s]','',text)   #remove punctuation
    text = re.sub(r'https?:\/\/.*[\r\n]*', ' ', text)   #remove hyperlinks
    text = re.sub(r'\b\w{1,3}\b', '',text) # remove words have length from 1 to 3
    text = re.sub(r'\b\w{11,20}\b', '',text) # remove words have length from 1 to 3
    return text

In [16]:
def text_sementic(text, method, rm_stop):
    text = text_clean(text)
    if rm_stop == True:
        filtered_tokens = [word for word in word_tokenize(text) if not word in set(stopwords.words('english'))]
        text = " ".join(filtered_tokens)

    if method == 'L':
        lemmer = WordNetLemmatizer()
        lemm_tokens = [lemmer.lemmatize(word) for word in word_tokenize(text)]
        return " ".join(lemm_tokens)

    #stemming
    if method == 'S':
        porter = PorterStemmer()
        stem_tokens = [porter.stem(word) for word in word_tokenize(text)]
        return " ".join(stem_tokens)
    if method == 'T':
      text = lemmatize_passage(text)

    return text

In [17]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lemmatize_passage(text):
    words = word_tokenize(text)
    pos_tags = pos_tag(words)
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]
    lemmatized_sentence = ' '.join(lemmatized_words)
    return lemmatized_sentence

In [18]:
def Words2Text(words):
  text = " ".join(words)
  return text;

In [19]:
def Text2Words(text):
  words = [word for word in word_tokenize(text)]
  return words

In [20]:
def data_w_tfidf_vectorizer(preprocessed_text, vocab = None):
    #vectorize dataset
    tfidf = TfidfVectorizer(stop_words='english',use_idf=True,vocabulary = vocab)
    vectorized_data = tfidf.fit_transform(preprocessed_text)
    return tfidf, vectorized_data

In [21]:
def InsertFeature(df_data,featurename, feature):
  df_res = df_data[['Mã số ID','count']].copy()
  df_res.loc[:,featurename] = feature.copy()

  df_res['effort'] = df_res['count'] / df_res.groupby('Mã số ID')['count'].transform('count')
  #df_res = pd.concat([df_res, df_data[['Mã số ID','count']].copy()])
  #df_res = pd.concat([df_res, feature.copy()])
  return df_res

In [22]:
def AddFeatureToAnswer(tfidf, original,dirt, dict_text):
  text = ""

  #df_reslabel = pd.DataFrame(columns=['grad'])
  feature_names = tfidf.get_feature_names_out()

  df_tkappend = pd.DataFrame()

  df_labelappend = pd.DataFrame(columns=['grad'])
  df_reslabel = pd.DataFrame(columns=['grad'])

  fexcel = ReadExcel(original,dirt)
  for idx, row in fexcel.iterrows():
    df_reslabel = pd.DataFrame(columns=['grad'])
    df_res = pd.DataFrame()
    df_ans = pd.DataFrame()

    path_row = row['content']
    print(path_row)
    df_ans = ReadCSV(original,row["content"])
    df_ans = AddCountField(df_ans)
    AssignLabel(df_ans)
    ftext = right_substring_to_char(path_row,'[',']')
    with open (os.path.join(original,ftext)) as content:
      text = content.read()
      #print(text)
      text = text_sementic(text, 'T', False)
      #print(text)
    #update dictionary
    if ftext not in dict_text.keys():
      dict_text[ftext] = text
    response = tfidf.transform([text])
    feature = response.toarray()[0]

    df_res = InsertFeature(df_ans,feature_names,feature)
    print(df_res.shape,df_reslabel.shape)
    print(df_res.head())
    #print(df_res.tail)
    df_reslabel['grad'] = df_ans['Q. 1 /10.00'].copy()
    df_tkappend = pd.concat([df_tkappend, df_res], ignore_index=True)
    df_labelappend = pd.concat([df_labelappend, df_reslabel], ignore_index=True)

  df_tkappend['Mã số ID'] = pd.to_numeric(df_tkappend['Mã số ID'])
  df_labelappend['grad'] = pd.to_numeric(df_labelappend['grad'])
  return df_tkappend, df_labelappend

In [23]:
def WeightedEffort(df):
  df['effort'] = df['count'] / df.groupby('Mã số ID')['count'].transform('count')

In [24]:
def ReadPreLabs(original,filepath):
  Labs_info = ReadExcel(original,filepath)
  df_prelabs = ReadLabFiles(original,Labs_info)  # add count
  AssignLabel(df_prelabs)                        # add label

In [25]:
def ModelData(vectorized_data, label_train, name_model, alpha = 0.1):
    print(vectorized_data.shape, label_train.shape)
    if name_model == 'Naive Bayes':
      print(name_model)
      #define model
      model = MultinomialNB(alpha=0.1)
      model.fit(vectorized_data, label_train)
    elif name_model == 'KNN':
      print(name_model)
    ## K-Nearest Neighbors
      model = KNeighborsClassifier(n_neighbors=3, p =1) #24
      model.fit(vectorized_data, label_train)
    ## name_model
    elif name_model == 'Random Forests':
      print(name_model)
      #model = RandomForestClassifier(max_depth=2, random_state=0)
      model = RandomForestClassifier(max_depth=3, min_samples_leaf=10, n_estimators=100, random_state=42)
      model.fit(vectorized_data, label_train)
    return model

In [26]:
def EvaluateModel(model, data_test, label_test):
    #evaluate model
    print(data_test.shape,label_test.shape)
    predictions = model.predict(data_test)

    #predictions = predictions.astype(int)
    #label_test = label_test.astype(int)

    # pred = pd.DataFrame(predictions.astype(int),columns=['Pred'])
    # pred.to_excel(os.path.join(path_prepost,"POST_LAB1/"+"OutputPrediction.xlsx"))
    # label_test.to_excel(os.path.join(path_prepost,"POST_LAB1/"+"OutputPredictionLabel.xlsx"))
    # print(predictions[0:5])

    accuracy = accuracy_score( label_test['grad'], predictions)
    balanced_accuracy = balanced_accuracy_score(label_test, predictions)
    precision = precision_score(label_test, predictions,average='weighted')

    print("Accuracy:",round(100*accuracy,2),'%')
    print("Balanced accuracy:",round(100*balanced_accuracy,2),'%')
    print("Precision:", round(100*precision,2),'%')
    return predictions

In [27]:
def BestKNN(df_train, df_label):
  k_values = [i for i in range (1,31)]
  scores = []

  scaler = StandardScaler()
  X = scaler.fit_transform(df_train)

  for k in k_values:
      knn = KNeighborsClassifier(n_neighbors=k)
      score = cross_val_score(knn, X, df_label, cv=5)
      scores.append(np.mean(score))
  sns.lineplot(x = k_values, y = scores, marker = 'o')
  plt.xlabel("K Values")
  plt.ylabel("Accuracy Score")

## Load data transform to vocabulary

In [28]:
dic_text = ReadTextFromExcel(path_prepost,"PRE_LAB1/PreLab01.xlsx")
print(len(dic_text))
dic_text.keys()

PRE_LAB1/Q 1.1.txt
PRE_LAB1/Q 1.2.txt
PRE_LAB1/Q 1.5.txt
PRE_LAB1/Q 2.7.txt
PRE_LAB1/Q 2.6.txt
PRE_LAB1/Q 3.1.txt
PRE_LAB1/Q 3.2.txt
PRE_LAB1/Q 3.3.txt
PRE_LAB1/Q 3.7.txt
PRE_LAB1/Q 4.2.txt
10


dict_keys(['PRE_LAB1/Q 1.1.txt', 'PRE_LAB1/Q 1.2.txt', 'PRE_LAB1/Q 1.5.txt', 'PRE_LAB1/Q 2.7.txt', 'PRE_LAB1/Q 2.6.txt', 'PRE_LAB1/Q 3.1.txt', 'PRE_LAB1/Q 3.2.txt', 'PRE_LAB1/Q 3.3.txt', 'PRE_LAB1/Q 3.7.txt', 'PRE_LAB1/Q 4.2.txt'])

In [29]:
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/lequochuy/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [33]:
#preprocessed_text = [text_sementic(text, 'S', False) for text in dic_text.values()]
preprocessed_text = [text_sementic(text, 'T', False) for text in dic_text.values()]

LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger_eng[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger_eng/[0m

  Searched in:
    - '/Users/lequochuy/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.8/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.8/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.8/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/Users/lequochuy/nltk_data'
**********************************************************************


In [ ]:
# Get Vectorize
tfidf, data_vector = data_w_tfidf_vectorizer(preprocessed_text)
print(tfidf.get_feature_names_out().shape, data_vector.shape)
tfidf.get_feature_names_out()

## Load answer Pre

In [ ]:
df_re, df_relabel = AddFeatureToAnswer(tfidf, path_prepost, "PRE_LAB1/PreLab01.xlsx", dic_text)

In [ ]:
print(df_re.head())

In [ ]:

n, bins = np.histogram(df_relabel)
labels = [1, 2, 3, 4,5,6,7,8,9,10]
bar_labels = ['red', 'blue', '_red', 'orange']
bar_colors = ['blue', 'orange', 'green', 'red','purple','brown','pink','gray','olive','cyan']
plt.bar(labels, n, label=labels, color=bar_colors)

#plt.set_ylabel('fruit supply')
#plt.set_title('Fruit supply by kind and color')
plt.legend(title='Grade')
plt.show()

## Load answer Post

In [ ]:
df_res, df_reslabel = AddFeatureToAnswer(tfidf,path_prepost,"POST_LAB1/PostLab01.xlsx",dic_text)
#WeightedEffort(df_res)
print(df_res.shape,df_reslabel.shape)
print(df_res.head())
print(df_reslabel.info())
df_reslabel.head()

## Predictions

## Naive Bayes model

In [ ]:
#name_model = 'Naive Bayes'
name_model = 'KNN'
#name_model = 'Random Forests'
model = ModelData(df_re, df_relabel, name_model,alpha = 0.1)

In [ ]:
BestKNN(df_re, df_relabel)

In [ ]:
#print(df_res.shape,df_reslabel.shape)
predictions = EvaluateModel( model, df_res, df_reslabel)

In [ ]:
cm = confusion_matrix(df_reslabel, predictions)

ConfusionMatrixDisplay(confusion_matrix=cm).plot();

## Select features

In [ ]:
ch2 = SelectKBest(chi2, k=22) #30
ch2.fit_transform(df_re, df_relabel)

In [ ]:
#feature_names = ['mssv','count']
print(df_re.columns)

In [ ]:
#feature_names = ['mssv','count']
#feature_names = feature_names.append(tfidf.get_feature_names_out())
feature_names = df_re.columns
#feature_names = ['mssv','count'].append(feature_names)
feature_names_chi = [feature_names[i] for i in ch2.get_support(indices=True)]

print(len(feature_names_chi))
#feature_names_chi = feature_names_chi.append(['count'])
print(feature_names_chi)
feature_names_chi = feature_names_chi[1:]

## FEATURES

In [ ]:
feature_names_chi = Words2Text(feature_names_chi)
feature_names_chi = text_sementic(feature_names_chi, 'T', False)

In [ ]:
feature_names_chi = Text2Words(feature_names_chi)
feature_names_chi

In [ ]:
tfidf, data_vector = data_w_tfidf_vectorizer(preprocessed_text,feature_names_chi)
print(tfidf.get_feature_names_out().shape)
tfidf.get_feature_names_out()

In [ ]:
df_Prereduce, df_Prereducelabel = AddFeatureToAnswer(tfidf,path_prepost,"PRE_LAB1/PreLab01.xlsx",dic_text)
#WeightedEffort(df_Prereduce)

In [ ]:
df_resreduce, df_resreducelabel = AddFeatureToAnswer(tfidf,path_prepost,"POST_LAB1/PostLab01.xlsx",dic_text)
#WeightedEffort(df_resreduce)

In [ ]:
print(df_resreduce.head())

In [ ]:
BestKNN(df_Prereduce, df_Prereducelabel)

In [ ]:
model = ModelData(df_resreduce, df_resreducelabel, name_model,alpha = 0.1)

In [ ]:
predictions = EvaluateModel( model, df_resreduce, df_resreducelabel)

In [ ]:
cm = confusion_matrix(df_reslabel, predictions)

ConfusionMatrixDisplay(confusion_matrix=cm).plot();